# Evaluation

In [1]:
import pandas as pd
import os
import re

df = pd.read_parquet("data/processed/colab/v2/raw_test.parquet")

### Example generating batchs samples

In [2]:
def id2text(text, movies):
    for movie_id in re.findall(r"@\d+", text):
        if movies[movies["movieId"] == movie_id[1:]].empty:
            movie_name = "<unk>"
        else:
            movie_name = movies[movies["movieId"] == movie_id[1:]]["movieName"].iloc[0]
        text = text.replace(movie_id, movie_name)
    return text

In [3]:
movies = pd.DataFrame(
    df["movieMentions"]
    .explode()
    .drop_duplicates()
    .dropna()
    .reset_index(drop=True)
    .tolist()
)

In [4]:
df_messages = df.explode("messages_translated")

df_messages["timeOffset"] = df_messages["messages_translated"].apply(
    lambda x: x["timeOffset"]
)
df_messages["text"] = df_messages["messages_translated"].apply(lambda x: x["text"])
df_messages["senderWorkerId"] = df_messages["messages_translated"].apply(
    lambda x: x["senderWorkerId"]
)
df_messages["messageID"] = df_messages["messages_translated"].apply(
    lambda x: x["messageId"]
)
df_messages.drop(
    columns=[
        "messages",
        "messageID",
        "messages_translated",
        "movieMentions",
        "respondentQuestions",
        "initiatorWorkerId",
        "initiatorQuestions",
        "timeOffset",
    ],
    inplace=True,
)

df_messages

,conversationId,respondentWorkerId,text,senderWorkerId
0,20001,957,"Olá, estou procurando por um filme como o @111...",956
0,20001,957,Você deveria assistir @151656.,957
0,20001,957,É um grande? Eu nunca vi isso. Eu já vi @192131.,956
0,20001,957,Eu quero dizer @134643,956
0,20001,957,"Sim, @151656 é muito engraçado e @94688 também.",957
...,...,...,...,...
1341,23322,1082,@177387,1082
1341,23322,1082,"Obrigado pelas suas recomendações, tenha um bo...",1084
1341,23322,1082,Foi um prazer compartilhar filmes com você. Te...,1082
1341,23322,1082,Adeus,1082


In [5]:
import re


def has_movie(text):
    return re.findall(r"@\d+", text)


def get_movie(list_movies, movies):
    movies_text = []
    for movie_id in list_movies:
        if movie_id == None:
            movies_text.append("")
        else:
            if movies[movies["movieId"] == movie_id[1:]].empty:
                movie_name = "<unk>"
            else:
                movie_name = movies[movies["movieId"] == movie_id[1:]][
                    "movieName"
                ].iloc[0]
            movies_text.append(movie_name)
    return movies_text

In [6]:
df_test = df_messages.copy()
df_test['has_movie'] = df_messages['text'].apply(lambda x: has_movie(x))
df_test['text'] = df_test['text'].apply(lambda x: id2text(x, movies))


In [7]:
df_test.query('conversationId == 20001')

,conversationId,respondentWorkerId,text,senderWorkerId,has_movie
0,20001,957,"Olá, estou procurando por um filme como o Supe...",956,[@111776]
0,20001,957,Você deveria assistir Police Academy (1984).,957,[@151656]
0,20001,957,É um grande? Eu nunca vi isso. Eu já vi Americ...,956,[@192131]
0,20001,957,Eu quero dizer American Pie (1999),956,[@134643]
0,20001,957,"Sim, Police Academy (1984) é muito engraçado ...",957,"[@151656, @94688]"
0,20001,957,Parece que eu preciso dar uma olhada neles.,956,[]
0,20001,957,"Sim, você vai gostar deles.",957,[]
0,20001,957,Eu agradeço seu tempo. Eu precisarei dar uma o...,956,[]
0,20001,957,Sim Lethal Weapon (1987),957,[@101794]
0,20001,957,"Obrigado, eu também vou assistir isso.",956,[]


In [19]:
def get_instructions(df):
    context_list = []
    instruction = "<|system|>\nVocê é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários.</s>"

    for conversation_id, group in df.groupby('conversationId'):
        msgs_list = [instruction]
        response = ""
        found = False
        
        for index, row in group.iterrows():
            if not found:
                if len(row['has_movie']) >= 2 and row['senderWorkerId'] == row['respondentWorkerId']:
                    context = ''.join(msgs_list)
                    response = row['text']
                    context_entry = {
                        "conversationId": conversation_id,
                        "instruction": context if context else None,
                        "expected_response": response,
                        "expected_movies": get_movie(row['has_movie'], movies),
                    }
                    context_list.append(context_entry)
                    found = True

            if row['senderWorkerId'] == row['respondentWorkerId']:
                msgs_list.append('\n<|assistant|>\n' + row['text'] + "</s>")
            else:
                msgs_list.append('\n<|user|>\n' + row['text'] + "</s>")

    return context_list


instructions = get_instructions(df_test)
instructions[0]


{'conversationId': 20001,
 'instruction': '<|system|>\nVocê é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários.</s>\n<|user|>\nOlá, estou procurando por um filme como o Super Troopers (2001).</s>\n<|assistant|>\nVocê deveria assistir Police Academy  (1984).</s>\n<|user|>\nÉ um grande? Eu nunca vi isso. Eu já vi American Pie .</s>\n<|user|>\nEu quero dizer American Pie  (1999)</s>',
 'expected_response': 'Sim, Police Academy  (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também.',
 'expected_movies': ['Police Academy  (1984)',
  'Police Academy 2: Their First Assignment (1985)']}

In [20]:
BASE_INSTRUCTION = "<|system|>\nVocê é um chatbot para indicação de filmes. Responda de maneira educada sugestões de filmes para os usuários.</s>"

print(f"Total contexts: {len(instructions)}")
print(f"Contexts with no previous messages: {sum(1 for entry in instructions if entry['instruction'] == BASE_INSTRUCTION)}")


Total contexts: 515
Contexts with no previous messages: 3


### Filtering data

In [ ]:
from string import punctuation
from tqdm import tqdm

dict_punctuation = {i: j for j, i in enumerate(punctuation)}

df_test = []

for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    df_explode = pd.DataFrame(row[["messages_translated"]].explode().tolist())
    # df_explode['text'] = df_explode.apply(lambda x: id2text(x['text'], movies), axis=1)
    worker_id = df_explode.iloc[0]["senderWorkerId"]
    instruction = ""
    response = ""

    changed = False

    for index, message in df_explode.iterrows():
        if changed == False:
            if message["senderWorkerId"] == worker_id:
                instruction += message["text"]
                if instruction[-1] not in dict_punctuation:
                    instruction += "."
            else:
                changed = True
                response += message["text"]
                if response[-1] not in dict_punctuation:
                    response += "."
        else:
            if message["senderWorkerId"] != worker_id:
                response += message["text"]
                if response[-1] not in dict_punctuation:
                    response += "."
            else:
                changed = False
                df_test.append({"initiator": instruction, "respondant": response})
                response = ""
                instruction = message["text"]
                if instruction[-1] not in dict_punctuation:
                    instruction += "."
df_test = pd.DataFrame(df_test)


# df_test.drop(['initiator', 'respondant'], axis=1, inplace=True)

In [ ]:
df_test["has_movie"] = df_test.apply(lambda x: has_movie(x["respondant"]), axis=1)
df_test["initiator"] = df_test.apply(lambda x: id2text(x["initiator"], movies), axis=1)
df_test["respondant"] = df_test.apply(
    lambda x: id2text(x["respondant"], movies), axis=1
)
df_test["has_movie"] = df_test.apply(
    lambda x: get_movie(x["has_movie"], movies), axis=1
)

In [ ]:
df_test

In [ ]:
df_test = df_test[df_test["has_movie"].str.len() != 0]

In [ ]:
df_test[df_test["has_movie"].str.len() > 3]

In [ ]:
pd.DataFrame(df_test, columns=["sample"]).to_parquet(
    "data/processed/colab/v2/train.parquet", index=False
)
pd.DataFrame(df_test, columns=["sample"]).to_parquet(
    "data/processed/colab/v2/test.parquet", index=False
)

In [ ]:
pd.read_parquet("data/processed/colab/v2/test.parquet").shape